# Bihar Elections — Case Study
This notebook provides a reproducible case study of the provided Bihar election dataset (`bihar_election_results.csv`).
Objectives:
- Load and inspect the dataset
- Clean and prepare data for analysis
- Explore party performance, seat counts, vote shares, and constituency-level insights
- Produce charts and short conclusions with next steps

## 1. Imports and setup

In [1]:
%pip install pandas numpy matplotlib seaborn plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
plt.style.use('seaborn')
%matplotlib inline
sns.set_context('talk')

OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

## 2. Load dataset and quick preview

In [ ]:
# Path (file is expected in the same folder as the notebook)
csv_path = 'bihar_election_results.csv'
df = pd.read_csv(csv_path)
print('Shape:', df.shape)
display(df.head())
print('
Columns:')
print(list(df.columns))
print('
Info:')
display(df.info())
print('
Missing values by column:')
display(df.isna().sum())

## 3. Basic cleaning and column normalization
The code below normalizes column names, guesses key columns (party, votes, constituency, candidate, year), and creates a `winner` flag when possible. It is adaptive so it will work when column names vary slightly.

In [ ]:
# Normalize column names to simple snake_case labels
def normalize_cols(cols):
    out = []
    for c in cols:
        s = str(c).strip().lower().replace(' ', '_').replace('-', '_')
        out.append(s)
    return out
df.columns = normalize_cols(df.columns)
print('Normalized columns:', list(df.columns))

# Heuristics to find key columns
col_text = ' '.join(df.columns)
c_party = next((c for c in df.columns if 'party' in c), None)
c_votes = next((c for c in df.columns if 'vote' in c and 'share' not in c), None)
c_votes_share = next((c for c in df.columns if 'vote_share' in c or 'vote% ' in c or 'vote_percent' in c), None)
c_const = next((c for c in df.columns if 'const' in c or 'constituency' in c or 'seat' in c), None)
c_candidate = next((c for c in df.columns if 'candidate' in c or 'name' in c), None)
c_position = next((c for c in df.columns if 'position' in c or 'rank' in c), None)
c_year = next((c for c in df.columns if 'year' in c), None)
c_status = next((c for c in df.columns if 'status' in c or 'result' in c or 'winner' in c), None)

print('Detected columns:')
print('party ->', c_party)
print('votes ->', c_votes)
print('votes_share ->', c_votes_share)
print('constituency ->', c_const)
print('candidate ->', c_candidate)
print('position/rank ->', c_position)
print('year ->', c_year)
print('status/result/winner ->', c_status)

# Create a winner flag if possible
if c_position is not None:
    df['winner'] = df[c_position].astype(str).str.replace(".0", "").astype(float) == 1
elif c_status is not None:
    s = df[c_status].astype(str).str.lower()
    df['winner'] = s.str.contains('win') | s.str.contains('winner') | s.str.contains('elected')
else:
    # fallback: assume the top row per constituency is the winner if votes column exists
    if c_votes is not None and c_const is not None:
        df['winner'] = False
        df_sorted = df.sort_values(by=[c_const, c_votes], ascending=[True, False])
        top_idx = df_sorted.groupby(c_const).head(1).index
        df.loc[top_idx, 'winner'] = True
    else:
        df['winner'] = False

# Ensure votes numeric when possible
if c_votes is not None:
    df[c_votes] = pd.to_numeric(df[c_votes], errors='coerce')

# Create a simplified 'party' column alias for downstream code
if c_party is None:
    # try common alternatives
    c_party = next((c for c in df.columns if 'affiliation' in c or 'alli' in c or 'party_name' in c), None)

if c_party is None:
    df['party'] = 'UNKNOWN'
else:
    df['party'] = df[c_party].astype(str).str.strip()

# make constituency column alias
if c_const is None:
    df['constituency'] = np.nan
else:
    df['constituency'] = df[c_const].astype(str).str.strip()

# year cast to int if exists
if c_year is not None:
    df['year'] = pd.to_numeric(df[c_year], errors='coerce').astype('Int64')

print('Prepared DataFrame sample:')
display(df.head())

## 4. Exploratory analysis
We'll calculate seats (winners), vote shares for parties, top parties by votes, and constituency-level summaries. The code is defensive to handle different dataset shapes.

In [ ]:
# Seats per party (count of winners)
if 'winner' in df.columns and df['winner'].any():
    seats = df[df['winner']].groupby('party').size().sort_values(ascending=False).rename('seats').reset_index()
    display(seats.head(20))
else:
    print('No winner flag found or no winners detected. Skipping seats calculation.')

# Total votes per party (when votes column present)
votes_col = next((c for c in df.columns if 'vote' in c and 'share' not in c), None)
if votes_col is not None:
    party_votes = df.groupby('party')[votes_col].sum().sort_values(ascending=False).rename('total_votes')
    party_votes = party_votes.reset_index()
    display(party_votes.head(20))
else:
    print('No votes column detected; cannot compute vote totals.')

# Vote share when vote share column or compute from totals
if 'vote_share' in df.columns or votes_col is not None:
    if 'vote_share' in df.columns:
        vs = df.groupby('party')['vote_share'].mean().sort_values(ascending=False).reset_index()
        display(vs.head(20))
    else:
        total_votes_all = df[votes_col].sum()
        party_votes['vote_share_pct'] = 100 * party_votes['total_votes'] / total_votes_all
        display(party_votes.head(20))

# Constituency-level top parties (if constituency exists)
if 'constituency' in df.columns and votes_col is not None:
    top_by_const = df.sort_values(votes_col, ascending=False).groupby('constituency').first().reset_index()
    top_counts = top_by_const['party'].value_counts().reset_index().rename(columns={'index':'party','party':'wins'})
    display(top_counts.head(20))
else:
    print('Skipping constituency-level top parties (missing constituency or votes column)')

## 5. Visualizations

In [ ]:
# Bar chart: seats (if available) or top parties by votes
fig, ax = plt.subplots(figsize=(12,6))
if 'seats' in globals():
    sns.barplot(data=seats.head(10), x='seats', y='party', palette='tab10', ax=ax)
    ax.set_title('Top 10 Parties by Seats (winners)')
else:
    if 'party_votes' in globals():
        sns.barplot(data=party_votes.head(10), x='total_votes', y='party', palette='tab10', ax=ax)
        ax.set_title('Top 10 Parties by Total Votes')
    else:
        ax.text(0.5,0.5,'No seats or vote totals available to plot', ha='center', va='center')
plt.tight_layout()
plt.show()

# Interactive pie or bar with plotly for vote share if available
if 'party_votes' in globals():
    try:
        fig = px.pie(party_votes.head(10), names='party', values='total_votes', title='Top 10 Parties: Vote Share')
        fig.show()
    except Exception as e:
        print('Plotly display failed:', e)

## 6. Time trends (if `year` present)

In [ ]:
if 'year' in df.columns and df['year'].notna().any():
    by_year = df.groupby('year').agg(total_votes=(votes_col, 'sum') if votes_col is not None else ('party','count'))
    by_year = by_year.reset_index()
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(by_year['year'], by_year.iloc[:,1], marker='o')
    ax.set_title('Total votes (or rows) by year')
    ax.set_xlabel('Year')
    ax.set_ylabel(by_year.columns[1])
    plt.tight_layout()
    plt.show()
else:
    print('No year column found or no year data present.')

## 7. Conclusions and next steps
- **Summary:** Use the outputs above to understand which parties dominate the dataset by votes or seats, and which parties win most constituencies. The notebook adapts to different column names and tries to compute seats and vote shares when data is available.
- **Limitations:** The dataset column names may vary; the notebook uses heuristics and may need manual column mapping for best results. If the CSV already contains a clear `winner` column and consistent `votes` numbers, results are more reliable.
- **Next steps:**
  - Map constituency winners to a geographical map (requires shapefile or geojson).
  - Clean party name disambiguation (alliance labels, synonyms).
  - Produce per-district swing and margin-of-victory analyses.

If you'd like, I can now: run this notebook, refine the column mappings for higher accuracy, or add specific analyses (e.g., gender breakdown, turnout heatmap).